In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
df = pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8.csv')

In [5]:
df = df[df['status'] != 'fp']
df = df[df['catalog'] != 'planck_no_z']
df.index = np.arange(len(df))

In [9]:
q_z = [df['z'].quantile(i/6) for i in range(0, 7)]
q_z[0] = 0
q_z[-1] = 2
q_m = [df['M500'].quantile(i/6) for i in range(0, 6)]
q_m[0] = 0
q_m[-1] = 20
print(repr(q_z))
print(repr(q_m))

[0, 0.146, 0.2769047021865845, 0.40646685, 0.545, 0.7208155989646913, 2]
[0, 1.5892898885458655, 1.9960987294061288, 2.3466809369916932, 2.8144785284579608, 20]


In [16]:
q_m500 = {}
for i in range(len(q_z) - 1):
    st = q_z[i]
    en = q_z[i + 1]
    matched = np.logical_and(st <= df['z'], df['z'] < en)
    cur_df = df[matched]
    q_m500[en] = [cur_df['M500'].quantile(i / 5) for i in range(6)]
    q_m500[en][-1] = 20


In [17]:
q_m500 = pd.DataFrame(q_m500)
q_m500

,0.146000,0.276905,0.406467,0.545000,0.720816,2.000000
0,0.009600,0.309300,0.324500,0.472700,0.958000,1.282935
1,0.613820,1.520560,1.796266,1.904794,1.949375,1.881850
2,1.272120,2.168034,2.288912,2.293961,2.278848,2.155825
3,1.903060,3.170817,2.877972,2.682044,2.655300,2.440612
4,2.736071,4.357869,4.906258,3.500986,3.318380,2.973040
5,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000


In [46]:
recalls = []
for j in range(len(q_m500) - 1):
    line = {}
    for i in range(len(q_z) - 1):
        z_st = q_z[i]
        z_en = q_z[i + 1]
        m_st = q_m500[z_en][j]
        m_en = q_m500[z_en][j + 1]
        
        matched1 = np.logical_and(z_st <= df['z'], df['z'] < z_en)
        matched2 = np.logical_and(m_st <= df['M500'], df['M500'] < m_en)
        cur_df = df[np.logical_and(matched1, matched2)]
        recall = np.count_nonzero(cur_df['status'] == 'tp') / len(cur_df)
        line['({:.2f},{:.2f})'.format(z_st, z_en)] = recall
    recalls.append(pd.DataFrame(line, index=[j]))
recalls = pd.concat(recalls)

In [47]:
recalls

,"(0.00,0.15)","(0.15,0.28)","(0.28,0.41)","(0.41,0.55)","(0.55,0.72)","(0.72,2.00)"
0,0.024390,0.092683,0.092683,0.068627,0.034146,0.034146
1,0.171569,0.191176,0.098039,0.078431,0.053659,0.034314
2,0.485294,0.326829,0.200000,0.132353,0.049020,0.039216
3,0.691176,0.803922,0.558824,0.245098,0.087805,0.087805
4,0.843902,0.921951,0.965854,0.731707,0.478049,0.243902


In [48]:
cm = sns.light_palette("green", as_cmap=True)

s = recalls.style.background_gradient(cmap=cm)
s

,"(0.00,0.15)","(0.15,0.28)","(0.28,0.41)","(0.41,0.55)","(0.55,0.72)","(0.72,2.00)"
0,0.024390,0.092683,0.092683,0.068627,0.034146,0.034146
1,0.171569,0.191176,0.098039,0.078431,0.053659,0.034314
2,0.485294,0.326829,0.200000,0.132353,0.049020,0.039216
3,0.691176,0.803922,0.558824,0.245098,0.087805,0.087805
4,0.843902,0.921951,0.965854,0.731707,0.478049,0.243902


In [45]:
z_st = 0.1
z_en=0.2
'({:.2f},{:.2f})'.format(z_st, z_en)

'(0.10,0.20)'

In [14]:
recalls = []
for j in range(len(q_m) - 1):
    line = {}
    m_st = q_m[j]
    m_en = q_m[j + 1]
    for i in range(len(q_z) - 1):
        z_st = q_z[i]
        z_en = q_z[i + 1]
        
        matched1 = np.logical_and(z_st <= df['z'], df['z'] < z_en)
        matched2 = np.logical_and(m_st <= df['M500'], df['M500'] < m_en)
        cur_df = df[np.logical_and(matched1, matched2)]
        recall = np.count_nonzero(cur_df['status'] == 'tp') / len(cur_df)
        line['({:.2f},{:.2f})'.format(z_st, z_en)] = recall
    recalls.append(pd.DataFrame(line, index=['({:.2f},{:.2f})'.format(m_st, m_en)]))
recalls = pd.concat(recalls)

In [15]:
cm = sns.light_palette("green", as_cmap=True)

s = recalls.style.background_gradient(cmap=cm)
s

,"(0.00,0.15)","(0.15,0.28)","(0.28,0.41)","(0.41,0.55)","(0.55,0.72)","(0.72,2.00)"
"(0.00,1.59)",0.152000,0.099099,0.052632,0.068493,0.022222,0.081633
"(1.59,2.00)",0.554745,0.194030,0.119205,0.062147,0.035714,0.017621
"(2.00,2.35)",0.673913,0.178218,0.143791,0.079365,0.059908,0.044444
"(2.35,2.81)",0.728155,0.298969,0.184049,0.156250,0.062500,0.066038
"(2.81,20.00)",0.863158,0.816631,0.744681,0.533520,0.318713,0.211321


In [22]:
nums = []
for j in range(len(q_m500) - 1):
    line = {}
    for i in range(len(q_z) - 1):
        z_st = q_z[i]
        z_en = q_z[i + 1]
        m_st = q_m500[z_en][j]
        m_en = q_m500[z_en][j + 1]
        
        matched1 = np.logical_and(z_st <= df['z'], df['z'] < z_en)
        matched2 = np.logical_and(m_st <= df['M500'], df['M500'] < m_en)
        cur_df = df[np.logical_and(matched1, matched2)]
        num = len(cur_df)
        line['({:.2f},{:.2f})'.format(z_st, z_en)] = num
    nums.append(pd.DataFrame(line, index=[j]))
nums = pd.concat(nums)

In [23]:
nums

,"(0.00,0.15)","(0.15,0.28)","(0.28,0.41)","(0.41,0.55)","(0.55,0.72)","(0.72,2.00)"
0,205,205,205,204,205,205
1,204,204,204,204,205,204
2,204,205,205,204,204,204
3,204,204,204,204,205,205
4,205,205,205,205,205,205


In [18]:
nums = []
for j in range(len(q_m) - 1):
    line = {}
    m_st = q_m[j]
    m_en = q_m[j + 1]
    for i in range(len(q_z) - 1):
        z_st = q_z[i]
        z_en = q_z[i + 1]
        
        matched1 = np.logical_and(z_st <= df['z'], df['z'] < z_en)
        matched2 = np.logical_and(m_st <= df['M500'], df['M500'] < m_en)
        cur_df = df[np.logical_and(matched1, matched2)]
        num = len(cur_df)
        line['({:.2f},{:.2f})'.format(z_st, z_en)] = num
    nums.append(pd.DataFrame(line, index=['({:.2f},{:.2f})'.format(m_st, m_en)]))
nums = pd.concat(nums)

In [19]:
nums

,"(0.00,0.15)","(0.15,0.28)","(0.28,0.41)","(0.41,0.55)","(0.55,0.72)","(0.72,2.00)"
"(0.00,1.59)",500,222,133,73,45,49
"(1.59,2.00)",137,134,151,177,196,227
"(2.00,2.35)",92,101,153,189,217,270
"(2.35,2.81)",103,97,163,224,224,212
"(2.81,20.00)",190,469,423,358,342,265
